# Converting `NetCDF` datasets to `GeoTiff`

<a id='libraries'></a>
## Python libraries

In [1]:
import numpy as np
import rioxarray
import xarray as xr
from google.cloud import storage

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path 
env_path = Path('.') / '.env'

<a id='utils'></a>
## Utils

In [3]:
def upload_blob(bucket_name, source_file_name, destination_blob_name, privatekey_path):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client.from_service_account_json(privatekey_path)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

In [4]:
def from_NetCDF_to_GeoTIFF(years):
    for year in years:
        print(f'Year: {year}')
        # Read NetCDF
        print(f'Reading NetCDF')
        if int(year) <= 2015:
            input_file_path = f'../data/dataset-satellite-land-cover/ESACCI-LC-L4-LCCS-Map-300m-P1Y-{year}-v2.0.7cds.nc'
        else:
            input_file_path = f'../data/dataset-satellite-land-cover/C3S-LC-L4-LCCS-Map-300m-P1Y-{year}-v2.1.1.nc'
            
        xds = xr.open_dataset(input_file_path)
        
        # Save GeoTIFF
        print(f'Saving GeoTIFF')
        output_file_path = f'../data/dataset-satellite-land-cover/ESA_{year}_ipcc.tif'
        data = xds["lccs_class"]
        data.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
        data.rio.set_crs("epsg:4326", inplace=True)
        data.rio.to_raster(output_file_path)
        
        # Upload GeoTIFF to Google cloud storage
        print(f'Uploading GeoTIFF to Google cloud storage')
        bucket_name = 'vizz-data-transfer'
        destination_blob_name = f'land-cover/ESA_{year}_ipcc.tif'

        upload_blob(bucket_name, output_file_path, destination_blob_name, os.getenv("PRIVATEKEY_PATH") )
        
        # Remove source file
        os.remove(output_file_path)      

## From `NetCDF` to `GeoTIFF` in Google cloud storage 

<a id='read_data'></a>
### Read `NetCDF`

In [5]:
year = '2017'
xds = xr.open_dataset(f'../data/dataset-satellite-land-cover/C3S-LC-L4-LCCS-Map-300m-P1Y-{year}-v2.1.1.nc')
xds

<xarray.Dataset>
Dimensions:              (bounds: 2, lat: 64800, lon: 129600, time: 1)
Coordinates:
  * lat                  (lat) float64 90.0 90.0 89.99 ... -89.99 -90.0 -90.0
  * lon                  (lon) float64 -180.0 -180.0 -180.0 ... 180.0 180.0
  * time                 (time) datetime64[ns] 2017-01-01
Dimensions without coordinates: bounds
Data variables:
    lccs_class           (time, lat, lon) uint8 ...
    processed_flag       (time, lat, lon) float32 ...
    current_pixel_state  (time, lat, lon) float32 ...
    observation_count    (time, lat, lon) uint16 ...
    change_count         (time, lat, lon) uint8 ...
    crs                  int32 ...
    lat_bounds           (lat, bounds) float64 ...
    lon_bounds           (lon, bounds) float64 ...
    time_bounds          (time, bounds) datetime64[ns] ...
Attributes:
    id:                         C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1
    title:                      Land Cover Map of 2017
    summary:                    This dataset characterizes the land cover of ...
    type:                       C3S-LC-L4-LCCS-Map-300m-P1Y
    project:                    EC C3S Land cover
    references:                 https://cds.climate.copernicus.eu/
    institution:                UCLouvain
    contact:                    copernicus-support@ecmwf.int
    comment:                    
    Conventions:                CF-1.6
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Standard Nam...
    keywords:                   land cover classification,satellite,observation
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Sc...
    license:                    EC C3S Land cover Data Policy
    naming_authority:           
    cdm_data_type:              grid
    TileSize:                   2025:2025
    tracking_id:                79c7ccd3-b2ae-4aab-9391-dab1163da910
    product_version:            2.1.1
    creation_date:              20190131T133214Z
    creator_name:               UCLouvain
    creator_url:                http://www.uclouvain.be/
    creator_email:              landcover-cci@uclouvain.be
    source:                     PROBA-V
    history:                    lc-sr-1.0, lc-classification-1.0,lc-user-tool...
    time_coverage_start:        20170101
    time_coverage_end:          20171231
    time_coverage_duration:     P1Y
    time_coverage_resolution:   P1Y
    geospatial_lat_min:         -90.0
    geospatial_lat_max:         90.0
    geospatial_lon_min:         null
    geospatial_lon_max:         null
    spatial_resolution:         300m
    geospatial_lat_units:       degrees_north
    geospatial_lat_resolution:  0.002778
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778

In [7]:
xds["lccs_class"].attrs

{'standard_name': 'land_cover_lccs',
 'flag_colors': '#ffff64 #ffff64 #ffff00 #aaf0f0 #dcf064 #c8c864 #006400 #00a000 #00a000 #aac800 #003c00 #003c00 #005000 #285000 #285000 #286400 #788200 #8ca000 #be9600 #966400 #966400 #966400 #ffb432 #ffdcd2 #ffebaf #ffc864 #ffd278 #ffebaf #00785a #009678 #00dc82 #c31400 #fff5d7 #dcdcdc #fff5d7 #0046c8 #ffffff',
 'long_name': 'Land cover class defined in LCCS',
 'valid_min': 1,
 'valid_max': 220,
 'ancillary_variables': 'processed_flag current_pixel_state observation_count change_count',
 'flag_meanings': 'no_data cropland_rainfed cropland_rainfed_herbaceous_cover cropland_rainfed_tree_or_shrub_cover cropland_irrigated mosaic_cropland mosaic_natural_vegetation tree_broadleaved_evergreen_closed_to_open tree_broadleaved_deciduous_closed_to_open tree_broadleaved_deciduous_closed tree_broadleaved_deciduous_open tree_needleleaved_evergreen_closed_to_open tree_needleleaved_evergreen_closed tree_needleleaved_evergreen_open tree_needleleaved_deciduous_clos

<a id='save_data'></a>
### Save `GeoTIFF`

In [ ]:
# Save GeoTIFF 
data = xds["lccs_class"]
data.rio.set_spatial_dims(x_dim='lon', y_dim='lat', inplace=True)
data.rio.set_crs("epsg:4326", inplace=True)
data.rio.to_raster(f'../data/dataset-satellite-land-cover/ESA_{year}_ipcc.tif')

### Upload `GeoTIFF` to Google cloud storage:

In [ ]:
bucket_name = 'vizz-data-transfer'
source_file_name = f'../data/dataset-satellite-land-cover/ESA_{year}_ipcc.tif'
destination_blob_name = f'land-cover/ESA_{year}_ipcc.tif'

upload_blob(bucket_name, source_file_name, destination_blob_name, os.getenv("PRIVATEKEY_PATH") )

### All together

In [ ]:
years = np.arange(2016, 2018+1, 1).astype(np.str)
from_NetCDF_to_GeoTIFF(years)

<a id='export_data'></a>
### Export `GeoTIFF` to Earth Engine

In [6]:
years = np.arange(2000, 2018+1, 1).astype(np.str)

# We create a collection asset in Earth Engine to use as a write target.
!earthengine create collection projects/soils-revealed/ESA_landcover_ipcc

# Next, we use the upload image command to send individual images to the collection.
for year in years:
    !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id projects/soils-revealed/ESA_landcover_ipcc/ESA_{year}_ipcc --pyramiding_policy=sample gs://vizz-data-transfer/land-cover/ESA_{year}_ipcc.tif 
        
        
        

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

W0519 11:19:36.971474 140735859868544 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/lib/python3.7/site-pac

### Export EE image to Asset

In [7]:
import ee
ee.Initialize()

In [12]:
years = np.arange(1983, 2017+1, 1).astype(np.str)


for year in years:
    image = ee.Image(f'users/iker/SOC_maps/SOC_concentration_q50/SOC_{year}')
    
    task = ee.batch.Export.image.toAsset(
        image = image,
        description = 'imageToAsset',
        assetId = f'projects/soils-revealed/experimental-dataset/SOC_concentration/SOC_concentration_{year}',
        scale = 300,
        pyramidingPolicy = {'b1': 'mean'},
        maxPixels = 1e13)
    task.start()

In [13]:
assets = ['SOCS_0_200cm_year_2010AD_10km', 'SOCS_0_200cm_year_NoLU_10km']

for asset in assets:
    image = ee.Image(f'users/iker/SOC_maps/{asset}')
    
    task = ee.batch.Export.image.toAsset(
        image = image,
        description = 'imageToAsset',
        assetId = f'projects/soils-revealed/Historic/{asset}',
        scale = 300,
        pyramidingPolicy = {'b1': 'mean'},
        maxPixels = 1e13)
    task.start()